
# Случайный лес


В данном случае мы говорим об **ансамбле** решающих деревьев (отсюда и слово лес). 
В обычных деревьях для задачи классификации в листях деревьев лежит класс. Для задачи регресии в листях лежит среднее значение ответов для всех объектов. 
Чтобы получить результат работы леса, ответы отдельных деревьев должны быть как-то аггрегированы. Вопрос: Как это можно удобно сделать?


In [ ]:
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_california_housing
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from tqdm.auto import trange
import category_encoders as ce

warnings.filterwarnings("ignore")

## Решающее дерево
Начнём с построения решающего дерева. Для демонстрации используем упрощенный датасет.

In [ ]:
california = fetch_california_housing()
california_X = pd.DataFrame(data=california.data, columns=california.feature_names)
california_Y = california.target
print(f"X shape: {california_X.shape}, Y shape: {california_Y.shape}")


X_train, X_test, y_train, y_test = train_test_split(
    california_X, california_Y, test_size=0.3, random_state=123, shuffle=True
)

**Задание**: Ниже пример подготовки нашего датасета с домами. Проведите все действия этого ноутбука и для него. 

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor

# TODO: обучите решающее дерево без ограничений на тренировочной выборке
tree = DecisionTreeRegressor()
tree.fit(X_train, y_train)

# TODO: рассчитайте MSE на тренировочной и тестовой выборках
print(f"MSE on train set: {mean_squared_error(y_train, tree.predict(X_train)):.2f}")
print(f"MSE on test set: {mean_squared_error(y_test, tree.predict(X_test)):.2f}")

Мы видим, что наше дерево сильно переобучилось - оно идеально предсказывает значения на трейне и довольно плохо - на тесте. Заглянем в эту проблему чуть глубже. 

Среднеквадратичный риск на фиксированной выборке X можно расписать как 
$$E = Var(h) + Bias^2(h) + Noise(y)$$
Здесь $Bias^2(h) = E_x[(\overline{h}(X) - \overline{y}(X))^2]$ показывает, насколько средняя модель отклонится от матожидания таргета (идеальной модели). 
$Var(h) = E_{x,D}[(h(X, D) - \overline{h}(X))^2]$ - показывает разброс обученных моделей относительно среднего ответа. 
$Noise(y) = E_{x,y}[(\overline{y}(X) - Y)^2]$ - дисперсия самого таргета при фиксированном x. Это неустранимая ошибка, которой соответствует самый идеальный прогноз.
Описание, как оно выводится, есть [здесь](https://getsomemath.ru/) и [здесь](https://education.yandex.ru/handbook/ml/article/bias-variance-decomposition). 

Смещение показывает, насколько хорошо можно с помощью данного семейства моделей приблизиться к оптимальной модели. Как правило, оно маленькое у сложных семейств и большое у относительно простых. Вопрос: Назовите такие семейства.

Дисперсия показывает, насколько будет изменяться предсказание в зависимости от выборки - то есть насколько ваше семейство склонно к переобучению. 

Существует известный пример сочетания смещения и разброса:

![bv](../additional/images/bv.png)


На нашем примере с домами посмотрим, какие bias и variance имеет наше дерево. 

In [ ]:
from mlxtend.evaluate import bias_variance_decomp

# TODO: выведите среднее смещение и среднюю дисперсию модели на тестовой выборке
avg_loss, avg_bias, avg_var = bias_variance_decomp(
    tree,
    np.array(X_train),
    np.array(y_train),
    np.array(X_test),
    np.array(y_test),
    loss="mse",
)
print(f"Average test loss: {avg_loss:.2f}")
print(f"Average test bias: {avg_bias:.2f}")
print(f"Average test variance: {avg_var:.2f}")

Декомпозиция довольно долгая, так как для вычисленя средних мы много раз (по умолчанию, 200) строим подвыборки тестового сета. 

Мы видим, что у нашего дерева высокая дисперсия и низкое смещение. Постараемся исправить это. Один из способов борьбы с переобучением – построение композиций моделей. На этом семинаре мы рассмотрим построение композиций при помощи бэггинга.

**Задание:**  Постройте графики зависимости смещения и разброса относительно глубины дерева, обученного на этом датасете.  

### Бэггинг
Бэггинг основан на идее бутстрапа. Бутстрап работает на удивление просто.
Предположим, что наша выборка D размера n на представляет генеральную совокупность. После этого мы можем сгененрировать эмпирическое распределение необходимой статистики, выбирая с замещением $N >> 100$ подвыборок объема n из этой совокупности (назовем псевдовыборками), и рассчитывая для них нужную статистику. 
![bag](../additional/images/bootstrap.png)


Суть алгоритма бэггинга:
1. Обучаем много деревьев на бутстрапированных подвыборках исходной выборки независимо друг от друга. Бутстрапированную подвыборку строим при помощи выбора $N$ (размер исходной выборки) наблюдений из исходной выборки с возвращением.
2. Усредняем предсказания всех моделей (например, берём арифметическое среднее).

Можно показать, что модель, построенная при помощи бэггинга, будет иметь **то же смещение**, что и у отдельных деревьев, но значительно **меньшую дисперсию** (если деревья нескоррелированы - именно этогно мы пытаемся достичь в RF, семплируя фичи).  В идеальном случае дисперсия уменьшится в N раз! На практике же мы такого вряд ли добьемся, но всек же улучшения ожидать стоит. 

Для начала обучим просто беггинг - он тоже есть в sklearn, и в него можно передать любые базовые модели.

In [ ]:
from sklearn.ensemble import BaggingRegressor

base_tree = DecisionTreeRegressor(random_state=123)

# TODO: обучите бэггинг с 20 деревьями, каждое из которых строится без ограничений
bag = BaggingRegressor(base_tree, n_estimators=20, n_jobs=4)
bag.fit(X_train, y_train)

# TODO: рассчитайте MSE на тренировочной и тестовой выборках
print(f"MSE on train set: {mean_squared_error(y_train, bag.predict(X_train)):.2f}")
print(f"MSE on test set: {mean_squared_error(y_test, bag.predict(X_test)):.2f}")

# TODO: выведите среднее смещение и среднюю дисперсию модели на тестовой выборке
avg_loss, avg_bias, avg_var = bias_variance_decomp(
    bag,
    np.array(X_train),
    np.array(y_train),
    np.array(X_test),
    np.array(y_test),
    loss="mse",
)
print(f"Average test loss: {avg_loss:.2f}")
print(f"Average test bias: {avg_bias:.2f}")
print(f"Average test variance: {avg_var:.2f}")

Как мы видим, по сравнению с единичным деревом смещение практически не изменилось, но дисперсия уменьшилась в несколько раз! Среднеквадратичная ошибка на тренировочной и тестовой выборках ближе, причем на тестовой уменьшилась, что говорит о том, что мы уменьшили переобучение единичного дерева.


### Случайный лес

При построении каждого дерева в бэггинге в ходе создания очередного узла будем выбирать случайный набор признаков, на основе которых производится разбиение. В результате такой процедуры мы уменьшим корреляцию между деревьями, за счёт чего снизим дисперсию итоговой модели. Такой алгоритм назвывается **случайным лесом** (Random Forest).

По сравнению с единичным деревом к параметрам случайного леса добавляются:
- `max_features` – число признаков, на основе которых проводятся разбиения при построении дерева.
- `n_estimators` – число деревьев.


Вопрос: Как можно задать `max_features`?

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# TODO: обучите случайный лес с 20 деревьями, каждое из которых строится без ограничений
rf = RandomForestRegressor(n_estimators=20, n_jobs=4)
rf.fit(X_train, y_train)

# TODO: рассчитайте MSE на тренировочной и тестовой выборках
print(f"MSE on train set: {mean_squared_error(y_train, rf.predict(X_train)):.2f}")
print(f"MSE on train set: {mean_squared_error(y_test, rf.predict(X_test)):.2f}")

# TODO: выведите среднее смещение и среднюю дисперсию модели на тестовой выборке
avg_loss, avg_bias, avg_var = bias_variance_decomp(
    rf,
    np.array(X_train),
    np.array(y_train),
    np.array(X_test),
    np.array(y_test),
    loss="mse",
)
print(f"Average test loss: {avg_loss:.2f}")
print(f"Average test bias: {avg_bias:.2f}")
print(f"Average test variance: {avg_var:.2f}")

Как мы видим, по сравнению с простым бэггингом смещение вновь осталось практически неизменным, а дисперсия немного уменьшилась. Конечно, если подобрать хорошие гиперпараметры, то получится снизить дисперсию ещё больше. 

Ошибка на тренировочной выборке увеличилась, а на тестовой — уменьшилась, что означает, что мы добились нашей цели в борьбе с переобученными деревьями!

**Дополнительное задание:** Постройте какую-нибудь несложную функцию и постройте на одном графике предсказания деревьев, обученных на выборках с добавленым шумом для такой функции.
Далее постройте такой же график, но для бэггингов, обученных на подвыборках. Для сравнения число семплов, которое вы выбирали для одного дерева (например, 50), должно остаться таким же для одного дерева в бэггинге. Для обучения бэггинга используйте больше семплов (50*n_trees). 
Ожидаемо вы увидете, что среднее предсказаний будет таким же (тот же Bias), но отдельные предсказания станут ближе к реальной зависимости (меньше Var).


## Особенности случайного леса
Много важных свойств леса описал в своём [блоге](https://www.stat.berkeley.edu/~breiman/RandomForests/cc_home.htm#remarks) Лео Бриман (Leo Breiman), создатель случайного леса. Рассмотрим часть из них.
Дополнительно: [статья от основ к практике](https://arxiv.org/pdf/1407.7502)
### Число деревьев и "Случайный лес не переобучается"
Cлучайный лес не переобучается именно с ростом числа деревьев (за счёт совместной работы бэггинга и использования случайных подпространств), но не в принципе. Посмотрим на поведение случайного леса при росте числа деревьев.

In [ ]:
n_trees = 100
train_loss = []
test_loss = []

for i in trange(1, n_trees, 2):
    rf = RandomForestRegressor(n_estimators=i, random_state=123, n_jobs=4)
    rf.fit(X_train, y_train)
    train_loss.append(mean_squared_error(y_train, rf.predict(X_train)))
    test_loss.append(mean_squared_error(y_test, rf.predict(X_test)))

plt.figure(figsize=(10, 7))
plt.title("Зависимость MSE от количества деревьев")
plt.grid()
plt.plot(train_loss, label="MSE_train")
plt.plot(test_loss, label="MSE_test")
plt.ylabel("MSE")
plt.xlabel("Number of trees")
plt.legend();

Как и ожидалось, по достижении некоторого числа деревьев обе ошибки практически не изменяются, то есть дальнейшего переобучения при росте числа деревьев не происходит. Поэтому на практике основное ограничение -бюджет на обучение.

При изменении какого-нибудь другого параметра на реальных данных переобучение может произойти. Например, случайный лес с ограниченными по глубине деревьями может предсказывать более точно, чем лес без ограничений. Примеры: [пример 1](https://datascience.stackexchange.com/questions/1028/do-random-forest-overfit), [пример 2](https://mljar.com/blog/random-forest-overfitting/).
Посмотрим, какой будет ошибка в зависимости от глубины.

**Задание**: Постройте график зависимости от глубины. Наблюдается ли переобучение?

In [ ]:
max_depth = 50
train_loss = []
test_loss = []
################
# YOUR CODE
################

**Задание**: Постройте график зависимости от числа признаков в сплите (max_features). Наблюдается ли переобучение?

Вопрос: Как думаете, какую глубину чтоит выбирать? А если вспомнить о bias и variance?

### 3.2. Out-of-bag-ошибка

Как мы обсудили выше, при построении случайного леса каждое дерево строится на бутстрапированной подвыборке, полученной из исходной обучающей выборки случайным набором с повторениями. В каждую подвыборку, таким образом, часть наблюдений попадет несколько раз, а часть ни разу. 

Какая вероятность, что семпл не попадет в подвыборку? Пусть в выборке $n$ наблюдений. Тогда вероятность семпла попасть в выборку с повторениями 1 раз - $1/n$. Не попасть - $1 - 1/n$. Бутстрап строит подвыборку размера n. Вероятность не попасть в подвыборку n раз тогда - $(1 - 1/n)^n$. Оценим асимптотику. При $ \lim_{n -> \inf}{(1 - 1/n)^n}$ сводится к замечательному пределу $(1 + 1/u)^u -> e $. 
В результате получаем $1/e \approx 0.37$.

А это значит, что мы можем получить оценку качества модели **бесплатно**! В случае MSE мы можем использовать MSE, а точности - error rate. Такая оценка называется **out-of-bag-ошибкой**. 

То, как конкретно она считается, зависит от реализации, но в целом алгоритм следующий: 
- Для каждого out-of-bag примера находим все модели, которые на нем не обучались.
- Голосованием определяем значение предсказание этих моделей. Считаем соответствующую оценку
- Считаем среднюю оценку на всех таких семплах.


In [ ]:
# oob_score_ = R2 на невиденных наблюдениях
rf = RandomForestRegressor(n_estimators=100, random_state=123, oob_score=True, n_jobs=4)
rf.fit(X_train, y_train)
rf.oob_score_

Вопрос: как OOB ошибка связана с оценкой на кросс-валидации?

Важно: при небольшом количестве деревьев оценка будет неточной. 

**Задание:** постройте график зависимости OOB ошибки для количества деревьев и их глубины.

### 3.3. Важность признаков
Как и решающие деревья, случайный лес позволяет оценивать важность признаков. Важность признаков можно оценивать, например, с помощью
**Gini Importance**.
Для одного дерева: при каждом разбиении по некоторому признаку, считаем Impurity decrease, взвешиваем по числу семплов в сплите и сохраняем результат. В конце суммируем.
Лес расширяет эту оценку: считается среднее важности признака по всем деревьям.
Однако тогда мы сталкиваемся с такой проблемой, что приоритет отдается признакам с большим числом уровней.

Более правильный способ (но и более затратный) -  **Permurtation importance**.
Для каждого дерева берется OOB семпл и считается количество голосов, отданных за правильный класс. Далее случайным образом перемешиваются значения переменной m в OOB и количество голосов, отданных за правильный класс, пересчитиывается.Далее одно вычитается из другого. Среднее  значение этой разницы по всем деревьям в лесу является необработанной оценкой важности для переменной m. 

In [ ]:
plt.figure(figsize=(10, 7))
plt.title("Feature importances")
plt.bar(california["feature_names"], rf.feature_importances_);

Однако такие оценки опасны в случае коррелирующих признаков. Посмотрим, что произойдёт с важностью, если добавить в выборку линейно зависимый признак.

In [ ]:
RM_mc = np.array((X_train.iloc[:, 5] * 2 + 3)).reshape(-1, 1)
X_train_new = np.hstack((X_train, RM_mc))

In [ ]:
rf.fit(X_train_new, y_train)

In [ ]:
plt.figure(figsize=(10, 7))
names = list(california["feature_names"])
names.append("_AveOccup")
plt.bar(names, rf.feature_importances_);

Важности перераспределились между линейной зависимыми признаками `AveOccup` и `_AveOccup`.  Поэтому такая оценка не обязательно четко отражает действительность. 

**Задание**: Посчитайте важности с помощью permutation importance.


In [ ]:
# from sklearn.inspection import permutation_importance

## Что еще

В библиотеке scikit-learn есть реализация ExtraTreesClassifier и ExtraTreesRegressor. Данный метод стоит использовать, если вы наблюдаете сильное переобучение на случайном лесе или градиентном бустинге.

Случайный лес не очень хорошо работает для сильно разреженных данных.

Как и одно дерево, RF не умеет экстраполировать.

Для данных, включающих категориальные переменные с различным количеством уровней, случайные леса предвзяты в пользу признаков с большим количеством уровней: когда у признака много уровней, дерево будет сильнее подстраиваться именно под эти признаки, так как на них можно получить более высокое значение информативности

RF хорошо параллелизуются, поэтому можно ускорить их обучение )
